# main_baseline_new
### The first part: using model: sklearn.linear_model.LogisticRegression
a unit test, which pull the first non-missing observation for each person for each temporal feature
and concatenate these to the static features and the corresponding mask matrix. 
The model is sklearn.linear_model.LogisticRegression to check whether the above predictor is useful.

In [71]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score
import pandas as pd
pd.set_option('display.max_columns', None)

In [29]:
org_data = np.load("./MIMIC_timeseries/24hours/series/imputed-normed-ep_1_24.npz", allow_pickle=True)
temporal_feature_dim = 12
static_feature_dim = 5
targets_dim = 21
adm_num = len(org_data['ep_tdata'])
temporal_feature = np.zeros((adm_num, temporal_feature_dim))
temporal_mask = np.zeros((adm_num, temporal_feature_dim))
static_feature = np.zeros((adm_num, static_feature_dim))
targets = np.zeros((adm_num, targets_dim))


In [30]:
for i in range(adm_num):
    for j in range(temporal_feature_dim):
        temp_f = org_data["ep_tdata"][i][:, j]
        temp_m = org_data["ep_tdata_masking"][i][:, j]
        if any(temp_m):
            temporal_feature[i, j] = temp_f[temp_m == True][0]
            temporal_mask[i, j] = 1
        else:
            temporal_feature[i, j] = 0
            temporal_mask[i, j] = 0
        

In [40]:
static_feature = org_data["adm_features_all"]
y_icd9 = org_data["y_icd9"]
y_mor = org_data["y_mor"]
targets = np.concatenate((y_icd9, y_mor), 1)
predictors = np.concatenate((static_feature, temporal_feature, temporal_mask), 1)
X_train, X_test, y_train, y_test = train_test_split(predictors, targets, test_size=0.33, random_state=42)

In [45]:
clf_models_list = []
for i in range(targets_dim):
    clf_models_list.append(LogisticRegression(max_iter=1000).fit(X_train, y_train[:, i]))



/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/shutingcui/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.

In [81]:
auprc_lists = np.zeros(targets_dim)
auroc = np.zeros(targets_dim)

for i in range(targets_dim):
    cur_y_true = y_test[:, i]
    cur_y_score = clf_models_list[i].predict_proba(X_test)[:,1] 
    # "y_score" used to compute auprc/auroc is the probability estimates of the positive class
    # through "clf_models_list[i].classes_" we know the order for each target class's label is [0 1]
    auprc_lists[i] = average_precision_score(cur_y_true, cur_y_score)
    auroc[i] = roc_auc_score(cur_y_true, cur_y_score)

In [73]:
auprc_lists

array([0.44669215, 0.23952274, 0.77001515, 0.47824311, 0.37871609,
       0.32696577, 0.94060067, 0.6018682 , 0.59582595, 0.70220011,
       0.05419829, 0.14603482, 0.23969321, 0.08384789, 0.41713585,
       0.10544544, 0.04146463, 0.50384245, 0.5575825 , 0.36984488,
       0.23998469])

In [74]:
baseline_for_auprc = np.zeros(21)
for i in range(targets_dim):
    baseline_for_auprc[i] = np.count_nonzero(y_test[:,i]) / len(y_test[:, i])
baseline_for_auprc

array([0.25238176, 0.17106159, 0.67786662, 0.36415447, 0.31753998,
       0.28317455, 0.82332426, 0.48205172, 0.38618578, 0.38618578,
       0.00425315, 0.10079959, 0.18747873, 0.0358115 , 0.31413746,
       0.08225587, 0.02943178, 0.44683566, 0.46444369, 0.33370194,
       0.10037428])

In [75]:
col_names = []
for i in range(1, 22):
    col_names.append("t"+str(i))
auprc_compares = np.zeros((2, targets_dim))
auprc_compares[0] = baseline_for_auprc
auprc_compares[1] = auprc_lists
auprc = pd.DataFrame(auprc_compares, columns=col_names, index = ['base', "logistic_model"])
auprc = auprc.round(3)
auprc

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
base,0.252,0.171,0.678,0.364,0.318,0.283,0.823,0.482,0.386,0.386,0.004,0.101,0.187,0.036,0.314,0.082,0.029,0.447,0.464,0.334,0.10
logistic_model,0.447,0.240,0.770,0.478,0.379,0.327,0.941,0.602,0.596,0.702,0.054,0.146,0.240,0.084,0.417,0.105,0.041,0.504,0.558,0.370,0.24


In [82]:
auroc_df = pd.DataFrame(auroc.reshape(1, targets_dim), columns=col_names)
auroc_df = auroc_df.round(3)
auroc_df

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
0,0.694,0.609,0.633,0.635,0.576,0.561,0.806,0.63,0.663,0.76,0.94,0.613,0.586,0.686,0.61,0.57,0.619,0.561,0.613,0.548,0.702


### The second part
Predictors are the same with the first part. The change is on the model, using pytorch linear function to see whether has the same result with the first part.

In [169]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, random_split, DataLoader
from torch.nn import BCELoss
from torch import optim

In [170]:
temporal_feature = temporal_feature.astype(np.float32)
static_feature = static_feature.astype(np.float32)
temporal_mask = temporal_mask.astype(np.float32)
targets = targets.astype(np.float32)

temporal_feature_tensor = torch.tensor(temporal_feature) # shape: [35623, 12]
temporal_mask_tensor = torch.tensor(temporal_mask) # shape: [35623, 12]
static_feature_tensor = torch.tensor(static_feature) # shape: [35623, 5]
targets_tensor = torch.tensor(targets) # shape: [35623, 21]
input_tensor = torch.cat((static_feature_tensor, temporal_feature_tensor, temporal_mask_tensor), 1) # [35623, 29]


In [171]:
class DatasetForBaselineModel(Dataset):
    def __init__(self, input_tensors, target_tensors):
        self.inputs = input_tensors
        self.targets = target_tensors
    def __len__(self):
        return(len(self.inputs))
    def __getitem__(self, idx):
        cur_input = self.inputs[idx]
        cur_target = self.targets[idx]
        return cur_input, cur_target

In [172]:
class LogisticRegression(nn.Module):
    def __init__(self, temporal_dim, mask_dim, static_dim, targets_dim):
        super(LogisticRegression, self).__init__()
        self.input_dim = temporal_dim + mask_dim + static_dim
        self.targets_dim = targets_dim
        self.logistic_layers = nn.ModuleList([nn.Sequential(nn.Linear(self.input_dim, 20), 
                                                            nn.ReLU(),
                                                            nn.Linear(20, 10),
                                                            nn.ReLU(),
                                                            nn.Linear(10, 1),
                                                            nn.Sigmoid()) 
                                              for t in range(self.targets_dim)])
        
    def forward(self, input_data):
        predict_list = torch.zeros(self.targets_dim)
        for i in range(self.targets_dim):
            predict_list[i] = self.logistic_layers[i](input_data)
        return predict_list   

In [173]:
dataset_baseline = DatasetForBaselineModel(input_tensor, targets_tensor)
train_size = int(0.7 * len(dataset_baseline))
test_size = len(dataset_baseline) - train_size
train_set, test_set = random_split(dataset_baseline, [train_size, test_size], 
                                   generator=torch.Generator().manual_seed(42))

In [174]:
batch_size = 10
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, drop_last=True)

### Multi-target Learning (21 binary labels)
* Currently taking the mean of each target's BCE loss as the training loss for the whole model

In [175]:
# training model
temporal_dim = 12
mask_dim = 12
static_dim = 5
targets_dim = 21
model = LogisticRegression(temporal_dim, mask_dim, static_dim, targets_dim)
criterion = BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.05)
model.train()
for epoch in range(20):
    print("epoch: \n", epoch)
    cum_loss_1000 = 0
    for t, train_batch in enumerate(train_loader):
        input_data = train_batch[0]
        target_data = train_batch[1]  # shape: [10, 21]
        predict_list = torch.zeros((batch_size, targets_dim))
        for i in range(batch_size):
            predict_list[i] = model(input_data[i])

        loss_all_targets = torch.zeros(targets_dim)
        for i in range(targets_dim):
            loss_all_targets[i] = criterion(predict_list[:, i], target_data[:, i])

        MTL_loss = torch.mean(loss_all_targets, 0)
        cum_loss_1000 += MTL_loss.item()
        optimizer.zero_grad()
        MTL_loss.backward()
        optimizer.step()
        if (t+1) % 1000 == 0:
            avg_loss = cum_loss_1000 / 1000
            print("current time: ", (t+1)*batch_size, " current averagetraining loss: ",  avg_loss)
            cum_loss_1000 = 0

epoch: 
 0
current time:  10000  current averagetraining loss:  7.075790196865797
current time:  20000  current averagetraining loss:  4.305454211711884
epoch: 
 1
current time:  10000  current averagetraining loss:  1.8672252172529697
current time:  20000  current averagetraining loss:  1.1549715019166469
epoch: 
 2
current time:  10000  current averagetraining loss:  0.5583356417417527
current time:  20000  current averagetraining loss:  0.548710470020771
epoch: 
 3
current time:  10000  current averagetraining loss:  0.5328971315324307
current time:  20000  current averagetraining loss:  0.5335102372169495
epoch: 
 4
current time:  10000  current averagetraining loss:  0.5255538522601128
current time:  20000  current averagetraining loss:  0.523465727865696
epoch: 
 5
current time:  10000  current averagetraining loss:  0.5215799546837807
current time:  20000  current averagetraining loss:  0.5204867178499699
epoch: 
 6
current time:  10000  current averagetraining loss:  0.51993399

In [177]:
model.eval()
y_true = torch.zeros((test_size, targets_dim))
y_score = torch.zeros((test_size, targets_dim))
for t, test_batch in enumerate(test_loader):
    cur_inputs = test_batch[0] # shape:[10, 29]
    cur_targets = test_batch[1] # shape: [10, 21]
    y_true[t*batch_size:(t+1)*batch_size, :] = cur_targets
    cur_predict = torch.zeros((batch_size, targets_dim))
    for i in range(batch_size):
        cur_predict[i] = model(cur_inputs[i])
    y_score[t*batch_size:(t+1)*batch_size, :] = cur_predict
    
auprc_list_pytorch_base = np.zeros(targets_dim)
auroc_list_pytorch_base = np.zeros(targets_dim)
for t_id in range(targets_dim):
    cur_y_true = y_true[:, t_id].detach().numpy()
    cur_y_score = y_score[:, t_id].detach().numpy()
    auprc_list_pytorch_base[t_id] = average_precision_score(cur_y_true, cur_y_score)
    auroc_list_pytorch_base[t_id] = roc_auc_score(cur_y_true, cur_y_score)

In [178]:
auprc_compares_pytorch = np.zeros((2, targets_dim))
auprc_compares_pytorch[0] = baseline_for_auprc
auprc_compares_pytorch[1] = auprc_list_pytorch_base
auprc_pytorch = pd.DataFrame(auprc_compares_pytorch, columns=col_names, index = ['base', "linear_model_pytorch"])
auprc_pytorch = auprc_pytorch.round(3)
auprc_pytorch

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
base,0.252,0.171,0.678,0.364,0.318,0.283,0.823,0.482,0.386,0.386,0.004,0.101,0.187,0.036,0.314,0.082,0.029,0.447,0.464,0.334,0.100
linear_model_pytorch,0.249,0.153,0.737,0.361,0.369,0.284,0.936,0.480,0.353,0.391,0.028,0.092,0.157,0.051,0.306,0.088,0.025,0.451,0.461,0.417,0.075


In [179]:
auroc_df_pytorch = pd.DataFrame(auroc_list_pytorch_base.reshape(1, targets_dim), columns=col_names)
auroc_df_pytorch = auroc_df_pytorch.round(3)
auroc_df_pytorch

,t1,t2,t3,t4,t5,t6,t7,t8,t9,t10,t11,t12,t13,t14,t15,t16,t17,t18,t19,t20,t21
0,0.499,0.484,0.622,0.501,0.571,0.497,0.792,0.501,0.476,0.501,0.934,0.474,0.43,0.646,0.49,0.522,0.43,0.501,0.501,0.545,0.371
